Initialization - import the necessary libararies

In [28]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import ast
import math

Import current directory

In [27]:
def get_project_root():
    """Returns project root folder."""
    try:
        current_dir = os.path.dirname(os.path.abspath(__file__))
        return os.path.dirname(os.path.dirname(current_dir))
    except NameError:
        return os.path.abspath(os.getcwd())

BASE_DIR = get_project_root()

Plot Actor Loss and Critic Loss each episode.

In [26]:
input_dir = os.path.join(BASE_DIR, 'loss')
output_dir = os.path.join(input_dir, "plot")
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

num_file = len([name for name in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, name))]) + 1

for i in range (1,num_file):
    filename = f'PPO Loss episode {i}.csv'
    filepath = os.path.join(input_dir, filename)

    df = pd.read_csv(filepath)
    
    fig,(ax1,ax2) = plt.subplots(1,2, figsize=(16,8))
    ax1.plot(df['Actor_loss'],'tab:orange')
    ax1.set_title("Actor Loss")
    ax2.plot(df['Critic_loss'],'tab:green')
    ax2.set_title("Critic Loss")
    ax1.set(xlabel='Steps',ylabel='loss')
    ax2.set(xlabel='Steps',ylabel='loss')
    fig.suptitle(f'PPO Update Iteration {i} Losses')
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.4, top= 0.85)
    plt.savefig(os.path.join(output_dir,f'loss_plot_per_ppo_update_iterate_{i}.png'))
    plt.clf()
    plt.close()


Calculate Average Actor and Critic Loss

In [29]:
input_dir = os.path.join(BASE_DIR, 'loss')
output_dir = os.path.join(input_dir, "avg_Actor_Critic_Loss")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

all_avg_actor = []
all_avg_critic = []

num_file = len([name for name in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, name))]) + 1

for i in range (1,num_file):
    filename = f'PPO Loss episode {i}.csv'
    filepath = os.path.join(input_dir, filename)

    df = pd.read_csv(filepath)
    avgLoss_actor = df['Actor_loss'].mean() 
    avgLoss_critic = df['Critic_loss'].mean()
    if (math.isnan(avgLoss_actor)):
        avgLoss_actor = 0.0
    if (math.isnan(avgLoss_critic)):
        avgLoss_critic = 0.0

    all_avg_actor.append(avgLoss_actor)
    all_avg_critic.append(avgLoss_critic)

data = {
    'Actor_Loss_avg': all_avg_actor,
    'Critic_Loss_avg': all_avg_critic
}
df = pd.DataFrame(data)
file_path = os.path.join(BASE_DIR, 'loss', 'avg_Actor_Critic_Loss', f'avg_loss.csv')
df.to_csv(file_path, index='False')


Plot Average Actor and Critic Loss

In [30]:
input_dir = os.path.join(BASE_DIR, 'loss', 'avg_Actor_Critic_Loss')
output_dir = os.path.join(input_dir, "plot_avg_acloss")
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

filename = f'avg_loss.csv'
filepath = os.path.join(input_dir, filename)

df = pd.read_csv(filepath)

window_size = 500

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
ax1.plot(df['Actor_Loss_avg'], color = 'blue', marker='^', markersize='4', linestyle='dashed', label='Average Actor Loss(Raw)')
ax1.plot(df['Actor_Loss_avg'].rolling(window=window_size).mean(), color = 'blue', marker='o', markersize='4', label='Average Actor Loss(500-ep Moving)')
ax1.set_title("Actor Loss", fontsize=16)
ax1.set_xlabel('Episodes', fontsize=14)
ax1.set_ylabel('Loss', fontsize=14)
ax1.legend()

ax2.plot(df['Critic_Loss_avg'], color = 'red', marker='+', markersize='4', linestyle='dashed', label='Average Critic Loss(Raw)')
ax2.plot(df['Critic_Loss_avg'].rolling(window=window_size).mean(), color = 'red', marker='D', markersize='4', label='Average Critic Loss(500-ep Moving)')
ax2.set_title("Critic Loss", fontsize=16)
ax2.set_xlabel('Episodes', fontsize=14)
ax2.set_ylabel('Loss', fontsize=14)
ax2.legend()

for ax in [ax1, ax2]:
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.yaxis.set_label_coords(-0.1, 0.5)  # Adjust y-axis label position
plt.savefig(os.path.join(output_dir,f'average_loss_plot.png'))
plt.clf()
plt.close()

Plot Overall and 500-episode moving average Rewards

In [31]:
input_dir = os.path.join(BASE_DIR, 'reward')
output_dir = os.path.join(input_dir, "plot")
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

filename = f'reward_plot.csv'
filepath = os.path.join(input_dir, filename)

df = pd.read_csv(filepath)

window_size = 500

plt.figure(figsize=(8,6))
plt.plot(df['Rewards'], color = 'blue', marker='o', markersize='4', label='raw rewards')
plt.plot(df['Rewards'].rolling(window=window_size).mean(), color = 'red', marker='^', markersize='2', label='500-ep moving average')

plt.xlabel('Reward')
plt.xlabel('Episode')
plt.title('REWARDS')
plt.legend()
plt.savefig(os.path.join(output_dir,f'reward_plot.png'))
plt.clf()
plt.close()

Plot the robot's coordination at each episode

In [32]:
input_dir = os.path.join(BASE_DIR, 'coordinates')
output_dir = os.path.join(input_dir, "plot")
image_path = os.path.join(BASE_DIR, 'images', "4_wheeled_robot_testing_obj_pes2.png")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

num_file = len([name for name in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, name))]) + 1

img = mpimg.imread(image_path)

def string_to_floats(s):
    try:
        # Safely evaluate the string as a Python literal
        value_list = ast.literal_eval(s)
        # Convert to floats and return as a Series
        return pd.Series([float(value_list[0]), float(value_list[1]), float(value_list[2]), float(value_list[3])])
    except:
        # Return NaN values if there's an error
        return pd.Series([float('nan'), float('nan'), float('nan'), float('nan')])

for i in range (1,num_file):
    filename = f'coordinates{i}_plot.csv'
    filepath = os.path.join(input_dir, filename)

    df = pd.read_csv(filepath)
    # print(df)
    df[['x_robot','y_robot','x_pedes','y_pedes']] = df['x_y'].apply(string_to_floats)

    # print (df['x'], df['y'])
    x_ro = df['x_robot']
    y_ro = df['y_robot']
    x_pe = df['x_pedes']
    y_pe = df['y_pedes']

    df['x_ro_path'] = (((x_ro - 0.85)/(3.58 - 0.85) * (1000 - 153)) + 153)
    df['y_ro_path'] = (((y_ro - (-0.179))/(-0.3 - (-0.179)) * (268 - 227)) + 227)
    df['x_pe_path'] = 741
    df['y_pe_path'] = (((y_pe - 0.21)/(-0.3 - 0.21) * (288 - 126)) + 126)

    x_ro_path = df['x_ro_path'].astype(int)
    y_ro_path = df['y_ro_path'].astype(int)
    x_pe_path = df['x_pe_path'].astype(int)
    y_pe_path = df['y_pe_path'].astype(int)

    plt.figure(figsize=(12,8))
    plt.plot(x_ro_path.values,y_ro_path.values ,'tab:blue' ,linewidth=5.0)
    plt.plot(x_pe_path.values,y_pe_path.values ,'tab:red'  ,linewidth=5.0)
    plt.imshow(img)
    plt.axis('off')

    plt.savefig(os.path.join(output_dir,f'coordinates{i}_plot.png'))
    plt.clf()
    plt.close()


Plot the robot's steering angle and distance-to-go at each episode

In [33]:
input_dir = os.path.join(BASE_DIR, "state")
output_dir = os.path.join(input_dir, "plot")
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

num_file = len([name for name in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, name))]) + 1

def string_to_floats(s):
    try:
        # Safely evaluate the string as a Python literal
        value_list = ast.literal_eval(s)
        # Convert to floats and return as a Series
        return pd.Series([float(value_list[0]), float(value_list[1]), float(value_list[2]), float(value_list[3]), float(value_list[4]), float(value_list[5]), 
                          float(value_list[6]), float(value_list[7]), float(value_list[8]), float(value_list[9]), float(value_list[10]), float(value_list[11]), 
                          float(value_list[12]),  float(value_list[13])])
    except:
        # Return NaN values if there's an error
        return pd.Series([float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan')])

for i in range (1,num_file):
    filename = f'transitions{i}_plot.csv'
    filepath = os.path.join(input_dir, filename)

    df = pd.read_csv(filepath)

    df[['x','y','dis','steer', 'velocity', 'dis_r_t', 'dis_r_o', 'dis_r_p','lane_l', 'lane_c', 'lane_r', 'box_detected', 'pedestrian_detected', 'robot_angle']] = df['State'].apply(string_to_floats)

    steer = round((df['steer'].astype(float) * 0.5),2)
    dqn_dist = df['dis_r_t'].reindex(index=df['dis_r_t'].index[:0:-1])
    dqn_tar_dist = 2.733 - (dqn_dist * 3.5)
    plt.figure(figsize=(16,8))
    fig,(ax1,ax2) = plt.subplots(1,2)
    ax1.plot(steer,'tab:orange')
    ax1.set_title("Steer Angle")
    ax2.plot(dqn_dist,'tab:green')
    ax2.set_title("Angular Distance")
    ax1.set(xlabel='Episode',ylabel='Steer Angle (radian)')
    ax2.set(xlabel='Episode',ylabel='Robot Distance (meters)')
    fig.suptitle(f'Robot Steer Angle and Distance Episode {i}')
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.3, top=0.85)
    plt.savefig(os.path.join(output_dir,f'steer_angle_episode {i}.png'))
    plt.clf()
    plt.close()

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>